## DRF Nested Routers

`drf-nested-routers` provides routers and fields to create nested resources in your API. This allows you to construct URLs that represent logical relationships, such as `/domains/{domain_pk}/nameservers/{pk}/`.

### Why use it?
Standard DRF routers are flat (`/users/`, `/posts/`). While flat APIs are often simpler, nested APIs can be more intuitive for resources that "belong" strictly to parents, providing natural filtering and scoping.

### Installation

```bash
pipenv install drf-nested-routers
```

No specific `INSTALLED_APPS` configuration is required.

### Basic Usage

To use nested routers, you define a parent router and then "attach" a child router to it.

#### 1. Define Routers in `urls.py`

```python
from rest_framework_nested import routers
from .views import DomainViewSet, NameserverViewSet

# 1. Create the parent router
router = routers.SimpleRouter()
router.register(r'domains', DomainViewSet)

# 2. Create the child router, nested under 'domains'
domains_router = routers.NestedSimpleRouter(router, r'domains', lookup='domain')
domains_router.register(r'nameservers', NameserverViewSet, basename='domain-nameservers')

urlpatterns = [
    path(r'', include(router.urls)),
    path(r'', include(domains_router.urls)),
]
```

This generates URLs like:
- `GET /domains/`
- `GET /domains/{domain_pk}/nameservers/`
- `GET /domains/{domain_pk}/nameservers/{pk}/`

### Accessing Parent IDs in ViewSets

In your child ViewSet (`NameserverViewSet`), the parent's primary key (e.g., `domain_pk`) is available in `self.kwargs`. You normally use this to filter the queryset.

```python
class NameserverViewSet(viewsets.ModelViewSet):
    serializer_class = NameserverSerializer

    def get_queryset(self):
        # The lookup kwarg is 'domain_pk' because we set lookup='domain' in the router
        return Nameserver.objects.filter(domain=self.kwargs['domain_pk'])
        
    def perform_create(self, serializer):
        # Automatically assign the parent when creating
        domain = Domain.objects.get(pk=self.kwargs['domain_pk'])
        serializer.save(domain=domain)
```

### Best Practices

1.  **Don't Nest Too Deep**: Avoid nesting more than one level deep (e.g., `/parents/1/children/2/toys/3/`). It becomes hard to maintain and consume. Flatten the API if possible.
2.  **Use `basename`**: Always specify a `basename` for nested routers to avoid naming conflicts in URL reversing.
3.  **Permissions**: Remember that access to a child resource typically implies read access to the parent. Ensure your permission classes handle this logic (e.g., checking if the user owns the `Domain` before letting them see `Nameservers`).